In [1]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ---------------- ----------------------- 6.3/14.9 MB 35.1 MB/s eta 0:00:01
   --------------------------------- ------ 12.3/14.9 MB 32.2 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 26.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install openai==0.28

  Attempting uninstall: openai
    Found existing installation: openai 1.52.0
    Uninstalling openai-1.52.0:
      Successfully uninstalled openai-1.52.0



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import openai
import pandas as pd
import numpy as np
import faiss
import pickle
import time
from dotenv import load_dotenv
import os
# OpenAI API 키 설정
load_dotenv(dotenv_path=".env")
openai.api_key = os.getenv("hana")
# 데이터 로드
faq_data_path = 'faq_data3.csv'  # FAQ 데이터 파일 경로
# 불필요한 줄바꿈 및 공백 제거

faq_data = pd.read_csv(faq_data_path)
faq_data['Question'] = faq_data['Question'].str.replace(r'Q.\r\n\s*질문\s*', '', regex=True).str.strip()
faq_data['Answer'] = faq_data['Answer'].str.replace(r'A.\r\n\s*답변\s*\n\n', '', regex=True).str.strip()

# OpenAI API로 배치 임베딩 생성 함수 정의
def get_openai_embeddings_batch(texts):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=texts
    )
    embeddings = [np.array(item['embedding'], dtype='float32') for item in response['data']]
    return embeddings

# 임베딩을 생성하고 저장하는 함수 정의 (배치 처리 및 캐싱 적용)
def generate_faq_embeddings_with_cache(data, column_name, cache_file):
    try:
        with open(cache_file, "rb") as f:
            embedding_cache = pickle.load(f)
    except FileNotFoundError:
        embedding_cache = {}

    texts_to_embed = [text for text in data[column_name] if text not in embedding_cache]
    print(f"임베딩 생성할 텍스트 수: {len(texts_to_embed)}")

    batch_size = 20  # 배치 크기 설정
    for i in range(0, len(texts_to_embed), batch_size):
        batch_texts = texts_to_embed[i:i + batch_size]
        try:
            batch_embeddings = get_openai_embeddings_batch(batch_texts)
            for text, embedding in zip(batch_texts, batch_embeddings):
                embedding_cache[text] = embedding
        except Exception as e:
            print("API 호출 중 오류 발생:", e)
            time.sleep(1)  # 잠시 대기 후 재시도

    # 임베딩 캐시 저장
    with open(cache_file, "wb") as f:
        pickle.dump(embedding_cache, f)

    return np.array([embedding_cache[text] for text in data[column_name]])

# FAQ 데이터에 대해 임베딩 생성 및 캐싱
faq_embeddings = generate_faq_embeddings_with_cache(faq_data, 'Question', "faq_embeddings_cache.pkl")

# faiss 인덱스 생성 및 벡터 추가
dimension = faq_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(faq_embeddings)

# 유사 문장 검색 함수 정의
def search_similar_sentence(question):
    question_embedding = get_openai_embeddings_batch([question])[0].reshape(1, -1)
    distances, indices = index.search(question_embedding, 1)
    return faq_data.iloc[indices[0][0]]['Answer']

print("임베딩 생성 및 검색이 완료되었습니다.")

# FAQ 데이터와 임베딩 저장
with open("faq_data2.pkl", "wb") as f:
    pickle.dump(faq_data, f)
with open("faq_embeddings2.pkl", "wb") as f:
    pickle.dump(faq_embeddings, f)

# 예제 질문
question = "금융인증서 갱신 방법은?"
print("질문:", question)
print("답변:", search_similar_sentence(question))


임베딩 생성할 텍스트 수: 406
임베딩 생성 및 검색이 완료되었습니다.
질문: 금융인증서 갱신 방법은?
답변: 하나원큐 앱 로그인 후 [메뉴 > 인증/보안 > 금융인증서 > 인증서 발급/재발급] 메뉴에서 금융인증서 발급을 해주세요. 금융인증서 발급을 위한 클라우드 생성 시 국내 통신사의 휴대전화번호(본인명의 휴대폰이 아니여도 됨)로 문자 또는 ARS인증을 수행해야 합니다.※ 이체회원이 아닌 경우 영업점 방문 또는 비대면 실명확인을 통해 이체회원 가입 후 금융인증서 발급 가능


In [ ]:
import openai
import faiss
import numpy as np
import pickle
from dotenv import load_dotenv
import os

# OpenAI API 키 설정
load_dotenv()
openai.api_key = os.getenv("hana")

# 저장된 FAQ 데이터와 임베딩 로드
with open("faq_data2.pkl", "rb") as f:
    faq_data_pd = pickle.load(f)
with open("faq_embeddings2.pkl", "rb") as f:
    faq_embeddings = pickle.load(f)

# faiss 인덱스 생성 및 임베딩 추가
dimension = faq_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(faq_embeddings)

def get_openai_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return np.array(response['data'][0]['embedding'], dtype='float32')


# 유사 문장 검색 함수 정의
def search_similar_sentence(question, threshold=0.5):
    # 질문 임베딩 생성
    question_embedding = get_openai_embedding(question).reshape(1, -1)
    distances, indices = index.search(question_embedding, 1)

    # FAQ 데이터에서 유사한 질문을 찾은 경우
    if distances[0][0] < threshold:
        return faq_data_pd.iloc[indices[0][0]]['Question'], faq_data_pd.iloc[indices[0][0]]['Answer'], True
    else:
        # 유사한 질문이 없는 경우 None 반환
        return None, None, False

# OpenAI GPT 모델을 사용해 추가 정보 생성
def generate_answer_with_gpt(question, related_question=None, related_answer=None):
    messages = [
        {"role": "system", "content": "당신은 친절한 정보 제공자입니다."}
    ]

    # FAQ에서 유사한 질문과 답변이 있는 경우, 이를 프롬프트에 추가
    if related_question and related_answer:
        messages.append({
            "role": "system",
            "content": f"관련 질문: {related_question}\n관련 답변: {related_answer}"
        })

    # 사용자 질문 추가
    messages.append({"role": "user", "content": question})

    # OpenAI API를 통해 답변 생성
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=300
    )
    return response['choices'][0]['message']['content']

# 최종 답변 생성 함수
def get_answer(question):
    related_question, related_answer, found = search_similar_sentence(question)
    if found:
        # FAQ 데이터에서 유사한 답변을 찾은 경우
        return generate_answer_with_gpt(question, related_question, related_answer)
    else:
        # FAQ 데이터에 없는 질문의 경우, 모델에게 질문만으로 답변 요청
        return generate_answer_with_gpt(question)

# 예제 질문
question = "암호화폐 투자에 대해 알려주세요."
print("질문:", question)
print("답변:", get_answer(question))

질문: 암호화폐 투자에 대해 알려주세요.
답변: 암호화폐는 디지털 형태의 자산으로, 블록체인 기술을 기반으로 하고 있습니다. 암호화폐는 중앙기관이나 정부의 간섭 없이 전 세계적으로 거래가 이루어질 수 있습니다. 암호화폐는 투자 수단으로 많이 사용되며, 많은 사람들이 가치 상승을 기대하고 가격의 변동성을 통해 수익을 얻고자 합니다.

암호화폐를 투자할 때 주의해야 할 점은 다음과 같습니다:
1. 시장의 변동성: 암호화폐 시장은 매우 변동성이 높으므로 손실을 줄이기 위해 리스크 관리가 필요합니다.
2. 보안: 개인정보와 자산을 안전하게 관리하기 위해 안전한 지갑을 사용하고 비밀번호를 잘 보호해야 합니다.
3. 정보 수집: 투자 전 암호화폐에


In [4]:
question = "하나원큐가 뭐야"
print("질문:", question)
print("답변:", get_answer(question))

질문: 하나원큐가 뭐야
답변: 하나원큐란 하나은행에서 제공하는 편리한 인증 방법 중 하나입니다. 하나원큐를 이용하면 간편하게 온라인 거래를 확인하고 서명할 수 있습니다. 하나원큐 앱을 통해 사용자 인증이 이루어지며, 전자서명 또는 간편인증 기능을 통해 다양한 온라인 거래를 보다 간편하게 처리할 수 있습니다. 이를 통해 보안성을 유지하면서도 사용자 편의성을 높일 수 있습니다.


In [5]:
question = "DSR이 뭐야."
print("질문:", question)
print("답변:", get_answer(question))

질문: DSR이 뭐야.
답변: DSR은 Debt Service Ratio(부채 상환 비율)의 약어로, 개인이나 가구가 소득에 비해 부담해야 하는 총 부채의 비율을 나타내는 지표입니다. DSR은 보통 월 소득 대비 월 상환 부담 금액을 나타내며, 이를 통해 해당 개인이나 가구가 현재 소득으로 얼마나 많은 부채를 상환할 수 있는지를 파악할 수 있습니다. 종종 대출 신청자의 신용평가나 대출 승인 여부를 결정할 때 DSR이 중요한 요소로 고려될 수 있습니다.


In [6]:
question = "금융인증서가 뭐야."
print("질문:", question)
print("답변:", get_answer(question))

질문: 금융인증서가 뭐야.
답변: 금융인증서는 인터넷 뱅킹을 할 때 사용하는 전자 인증 수단입니다. 이는 은행이나 금융기관에서 발행되며, 사용자를 인증하고 전자 금융거래를 보호하는데 사용됩니다. 보통은 은행에서 발급받으며, 안전한 결제와 송금, 거래내역 조회, 계좌조회 등의 업무에 필요한 전자서명 인증서입니다. 일반적으로 인터넷 뱅킹을 사용하려면 금융인증서가 필요합니다.


In [7]:
question = "금융 인증서 갱신 방법은 뭐야"
print("질문:", question)
print("답변:", get_answer(question))

질문: 금융 인증서 갱신 방법은 뭐야
답변: 금융인증서를 갱신하려면 하나원큐 앱에 접속하여 [메뉴 > 인증/보안 > 금융인증서 > 인증서 재발급] 메뉴에서 갱신할 수 있습니다. 갱신 시에도 국내 통신사의 휴대전화번호로 문자 또는 ARS 인증을 수행해야 합니다. 만약에 문제가 발생한다면 영업점을 방문하셔서 도움을 받을 수도 있습니다.


In [ ]:
import openai
import pandas as pd
import random
from dotenv import load_dotenv
import os

# OpenAI API 키 설정
load_dotenv()
openai.api_key = os.getenv("hana")

# 예시 금융 관련 추가 질문 리스트 (임의로 추가하여 100개의 질문을 준비)
additional_finance_questions = [
    "금융 투자는 어떻게 시작하나요?",
    "펀드는 어떻게 운영되나요?",
    "주식의 배당 수익은 어떻게 계산하나요?",
    "채권이란 무엇이며 어떻게 투자하나요?",
    "암호화폐는 안전한 투자 수단인가요?",
    "경제 지표가 주식 시장에 미치는 영향은?",
    "외환 거래에서 리스크를 줄이는 방법은?",
    "공매도란 무엇인가요?",
    "주식 분할의 장단점은 무엇인가요?",
    "인덱스 펀드와 액티브 펀드의 차이점은?",
    "금 가격 변동의 주요 원인은 무엇인가요?",
    "ETF와 뮤추얼펀드의 차이점은?",
    "암호화폐 지갑의 종류와 차이점은?",
    "포트폴리오 다각화의 중요성은?",
    "주식시장과 채권시장의 관계는?",
    "원자재 투자는 어떻게 이루어지나요?",
    "ESG 투자의 개념과 중요성은?",
    "금융 레버리지 효과란 무엇인가요?",
    "기술적 분석과 기본적 분석의 차이점은?",
    "외환 보유고의 역할은 무엇인가요?",
    "비트코인과 이더리움의 차이점은?",
    "신용등급이 투자에 미치는 영향은?",
    "연금 투자 전략은 무엇인가요?",
    "인플레이션에 대비하는 투자 방법은?",
    "리스크와 수익성의 관계는?",
    "대체 투자란 무엇인가요?",
    "달러 강세가 신흥 시장에 미치는 영향은?",
    "고정 금리와 변동 금리의 차이점은?",
    "퀀트 투자란 무엇인가요?",
    "헤지펀드와 뮤추얼펀드의 차이점은?",
    "경기 순환주와 비경기 순환주의 차이점은?",
    "환율 변동이 경제에 미치는 영향은?",
    "금융 파생상품의 종류는?",
    "경제 위기에 대비하는 투자 전략은?",
    "현금흐름할인법(DCF)의 원리는?",
    "주가수익비율(PER)의 의미는?",
    "적립식 투자와 일시불 투자의 장단점은?",
    "금리 인상이 자산 가격에 미치는 영향은?",
    "대출 금리와 예금 금리의 차이는?",
    "IPO란 무엇이며 어떻게 이루어지나요?",
    "배당 수익률이 중요한 이유는?",
    "기업 분석 시 고려해야 할 지표는?",
    "포트폴리오 리밸런싱이란?",
    "주식시장과 부동산 시장의 차이점은?",
    "안전자산과 위험자산의 구분 기준은?",
    "비트코인 채굴의 원리는?",
    "투자와 투기의 차이는?",
    "경기 침체에 대비하는 자산 배분 전략은?",
    "ESG 펀드의 장점은?",
    "리스크 헤지란 무엇인가요?"
]

# FAQ 데이터에 있는 질문을 포함하여 총 100개의 질문으로 만듭니다
faq_questions = faq_data['Question'].tolist()
sampled_faq_questions = random.sample(faq_questions, 50)
all_questions = sampled_faq_questions + additional_finance_questions
sampled_all_questions = random.sample(all_questions, 50)

# 모든 질문에 대해 답변 생성
all_data = []
for question in sampled_all_questions[:50]:  # 최대 100개만 사용
    answer = get_answer(question)
    all_data.append({"Question": question, "Answer": answer})

# 데이터프레임으로 변환
df = pd.DataFrame(all_data)

# 엑셀 파일로 저장
output_filename = "finance_questions_and_answers2.xlsx"
df.to_excel(output_filename, index=False)

print(f"모든 질문과 답변이 '{output_filename}' 파일에 저장되었습니다.")

모든 질문과 답변이 'finance_questions_and_answers2.xlsx' 파일에 저장되었습니다.


In [14]:
question = "금융인증서갱신방법을 알려줘"
print("질문:", question)
print("답변:", get_answer(question))

질문: 금융인증서갱신방법을 알려줘
답변: 금융인증서 갱신은 보통 매년 이루어지며, 하나원큐 앱을 통해 간단하게 처리할 수 있습니다. 아래 단계를 따라 진행해보세요.

1. 하나원큐 앱에 로그인합니다.
2. [메뉴 > 인증/보안 > 금융인증서] 에서 금융인증서 메뉴로 이동합니다.
3. "인증서 발급/재발급" 메뉴를 선택합니다.
4. 안내에 따라 인증 절차를 완료합니다.
5. 금융인증서 갱신이 완료되었습니다.

갱신 과정에서 어려움이 있는 경우, 해당 은행 또는 고객센터로 문의하시면 도와드릴 것입니다.
